In [110]:

%matplotlib inline
import tensorflow as tf
import os
from keras.backend import set_session
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import np_utils
from keras.callbacks import EarlyStopping 
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import optimizers

In [111]:

def create_session(gpu_id='0', pp_mem_frac=None):

    tf.reset_default_graph()
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id # can multiple?
    with tf.device('/gpu:' + gpu_id):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        if pp_mem_frac is not None:
            config.gpu_options.per_process_gpu_memory_fraction=pp_mem_frac
        session = tf.Session(config = config)
    return session
gpu_id = '0'
sess = create_session(gpu_id)
set_session(sess)


In [112]:
#讀取檔案+整理成可丟進ann的格式
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [113]:
#把label改成one hot編碼
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [114]:
#加入early stop 避免model overfitting
from keras.callbacks import EarlyStopping 
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=60, 
    verbose=2, 
    mode='min'
)

In [115]:
model = Sequential()
model.add(Dense(160, input_dim=784,activation ="relu"))
model.add(Dense(80,activation ="relu"))
model.add(Dense(40,activation ="relu"))
model.add(Dense(20,activation ="relu"))
model.add(Dense(10,activation ="softmax"))
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, clipvalue=1.)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

In [116]:
model.fit(x_train, y_train,validation_split=0.1, epochs=300,batch_size = 100,callbacks=[early_stopping])

Train on 54000 samples, validate on 6000 samples
Epoch 1/300
54000/54000 [==============================] - 2s 34us/step - loss: 2.7460 - acc: 0.3058 - val_loss: 1.4061 - val_acc: 0.4627
Epoch 2/300
54000/54000 [==============================] - 2s 31us/step - loss: 0.8305 - acc: 0.7483 - val_loss: 0.3301 - val_acc: 0.9033
Epoch 3/300
54000/54000 [==============================] - 2s 31us/step - loss: 0.3560 - acc: 0.9022 - val_loss: 0.2647 - val_acc: 0.9313
Epoch 4/300
54000/54000 [==============================] - 2s 31us/step - loss: 0.2661 - acc: 0.9268 - val_loss: 0.2070 - val_acc: 0.9442
Epoch 5/300
54000/54000 [==============================] - 2s 30us/step - loss: 0.2161 - acc: 0.9410 - val_loss: 0.1835 - val_acc: 0.9483
Epoch 6/300
54000/54000 [==============================] - 2s 30us/step - loss: 0.1812 - acc: 0.9498 - val_loss: 0.1637 - val_acc: 0.9558
Epoch 7/300
54000/54000 [==============================] - 2s 30us/step - loss: 0.1588 - acc: 0.9551 - val_loss: 0.1568 - v

54000/54000 [==============================] - 2s 39us/step - loss: 0.0049 - acc: 0.9990 - val_loss: 0.2270 - val_acc: 0.9695
Epoch 60/300
54000/54000 [==============================] - 2s 39us/step - loss: 0.0049 - acc: 0.9990 - val_loss: 0.2232 - val_acc: 0.9698
Epoch 61/300
54000/54000 [==============================] - 2s 39us/step - loss: 0.0049 - acc: 0.9990 - val_loss: 0.2257 - val_acc: 0.9693
Epoch 62/300
54000/54000 [==============================] - 2s 39us/step - loss: 0.0048 - acc: 0.9990 - val_loss: 0.2263 - val_acc: 0.9693
Epoch 63/300
54000/54000 [==============================] - 2s 39us/step - loss: 0.0048 - acc: 0.9990 - val_loss: 0.2269 - val_acc: 0.9683
Epoch 64/300
54000/54000 [==============================] - 2s 39us/step - loss: 0.0049 - acc: 0.9989 - val_loss: 0.2315 - val_acc: 0.9687
Epoch 65/300
54000/54000 [==============================] - 2s 39us/step - loss: 0.0047 - acc: 0.9990 - val_loss: 0.2318 - val_acc: 0.9693
Epoch 66/300
54000/54000 [==============

In [117]:
score = model.evaluate(x_test, y_test)
print("loss", score[0])
print("accuracy", score[1])

10000/10000 [==============================] - 0s 42us/step
loss 0.2868611091341993
accuracy 0.9656
